_**Monte Carlo Simulations are like Unit tests for Bandit Algorithms:-**_

- Bandit algorithms have to actively select which data you should acquire and analyze that data in real time. Indeed, bandit algorithms exemplify one of the two types of learnings that are not present in standard ML examples: active learning, which refers to algorithms that actively select which data they should receive; and online learning, which refers to algorithms that analyze data in real-time and provide results on the fly.
- The behavior of the algorithm depends on the data it sees, but the data the algorithm sees depends on the behavior of the algorithm.

- A Monte Carlo simulation will let out implementation of a bandit algorithm actively make decisions about which data it will receive, because our simulations will be able to provide simulated data in real-time to the algorithm for analysis.

- In short, we are going to deal with the feedback cycle by coding up both our bandit algorithm and a simulation of the bandit's arms that the algorithm has to select between.
- The two pieces of code then work together to generate an example of how the algorithm might really function in production.


_**Optimizing click-through rates**_
- Every time we show someone an ad, we will imagine that thers's a fixed probability that they will click on the ad. The bandit algorithm will then estimate this probability and try to decide on a strategy for showing ads that maximizes the click-through rate.

_**Conversion rates for new users**_
- Every time a new visitor comes to our site who is not already a registered user, we will imagine that there's a fixed probability that they will register as a user after seeing the landing page. We will then estimate this probability and try to decide on a strategy for maximizing our conversion rate.


In [1]:
import numpy
import random
import math
import pandas as pd
import matplotlib.pyplot as plt
from plotnine import *

import nbimporter
import EpsilonGreedy
import SoftMax
import UpperConfidenceBound

In [2]:
class BernoulliArm():
    '''
        Simulating hypothetical arm
        BernoulliArm :- An arm that rewards you with a value of 1 some percentage of the time and rewards you with a value of 0 
        the rest of the time ( 1 - x )
    '''
    def __init__(self, p):
        self.p = p
        
    def __repr__(self):
        return( 'BernoulliArm probability: {0:.2f}'.format(self.p) )
    
    def draw(self):
        if random.random() > self.p:
            return 0.0 
        else:
            return 1.0
    
BernoulliArm(0.2)

BernoulliArm probability: 0.20

In [3]:
def testing_algorithm(  algo, arms, num_sims=5000, horizon=250 ):
        '''
            algo : A bandit algorithm we want to test ( epsilon-greedy, UCB, Softmax, ...)
            arms : An array of arms we want to simulate draws from
            num_sims : A fixed number of simulations to run to average over the noise in each simulation
            horizon : The number of times each algorithm is allowed to pull on arms during each simulation. Any algorithm that's
            not terrible will eventually learn which arm is best; the interesting thing to study in a simulation is whether
            an algorithm does well when it only has 100 ( or 100k ) tries to find the best arm
        '''
        chosen_arms = [ 0.0 for i in range(num_sims * horizon) ] 
        rewards = [ 0.0 for i in range( num_sims *horizon)]
        cumulative_rewards = [ 0.0 for i in range(num_sims * horizon) ]
        sim_nums = [ 0.0 for i in range(num_sims*horizon)]
        times = [ 0.0 for i in range(num_sims*horizon)]
        
        for sim in range( num_sims ):
            sim += 1 # because range generates numbers from 0
            algo.initialize( len(arms) )
            for t in range( horizon) :
                t += 1
                index = ( sim - 1 ) * horizon + t - 1 
                
                sim_nums[index] = sim
                times[index] = t
                
                chosen_arm = algo.select_arm() # draws based on epsilon
                chosen_arms[index] = chosen_arm
                
                reward = arms[chosen_arms[index]].draw() # draws based on success rate
                rewards[index] = reward
                
                if t == 1 :
                    cumulative_rewards[index] = reward
                else:
                    cumulative_rewards[index] = cumulative_rewards[index-1] + reward
                    
                algo.update( chosen_arm, reward)
        return [ sim_nums, times, chosen_arms, rewards, cumulative_rewards]